In [7]:
import numpy as np
import pandas as pd
import evaluate
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [8]:
datasetraw = load_dataset("csv",data_files="data/climate_fever_labels.csv") 
datasetraw

Generating train split: 1535 examples [00:00, 40698.23 examples/s]


DatasetDict({
    train: Dataset({
        features: ['claim_label', 'votes'],
        num_rows: 1535
    })
})

In [9]:
dataset = datasetraw.rename_columns({'votes':'text', 'claim_label':'label'})
dataset = dataset["train"].train_test_split(test_size=0.2)
dataset.column_names

{'train': ['label', 'text'], 'test': ['label', 'text']}

In [10]:
model_name = 'distilbert-base-uncased'


tokenizer = AutoTokenizer.from_pretrained(model_name)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

c:\Users\enzol\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 307/307 [00:00<00:00, 2655.91 examples/s]


In [11]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4)
training_args = TrainingArguments(output_dir="test_trainer",num_train_epochs=1)
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
trainer.train()

KeyboardInterrupt: 

In [ ]:
trainer.evalute()